In [1]:
import tensorflow

c:\python39\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\python39\lib\site-packages\numpy\.libs\libopenblas.NOIJJG62EMASZI6NYURL6JBKM4EVBGM7.gfortran-win_amd64.dll
c:\python39\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense
import numpy as np

In [3]:
batch_size = 64
epochs = 100
latent_dim = 256
num_samples = 20000

In [4]:
data_path = 'fra.txt'

In [5]:
input_texts = []
output_texts = []
input_characters = set()
output_characters = set()

with open(data_path, 'r', encoding = 'utf-8') as f:
    lines = f.read().split('\n')
for line in lines[:min(num_samples,len(lines)-1)]:
    input_text, output_text, _ = line.split('\t')
    
    # We use 'tab' as the 'start sequence' character
    # for the targets, and '\n' as the 'end sequence' character.
    
    output_text = '\t' + output_text + '\n'
    input_texts.append(input_text)
    output_texts.append(output_text)

    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in output_text:
        if char not in output_characters:
            output_characters.add(char)


In [6]:
len(output_characters)

101

In [7]:
input_characters = sorted(list(input_characters))
output_characters = sorted(list(output_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(output_characters)
max_encoder_seq_length = max([len(text) for text in input_texts])
max_decoder_seq_length = max([len(text) for text in output_texts])

In [8]:
print("Number of Samples:", len(input_texts))
print('Number of unique input Tokens:' , num_encoder_tokens)
print('Number of unique output Tokens:' , num_decoder_tokens)
print('Max sequence length for inputs:' , max_encoder_seq_length)
print('Max sequence length for outputs:' , max_decoder_seq_length)

Number of Samples: 20000
Number of unique input Tokens: 74
Number of unique output Tokens: 101
Max sequence length for inputs: 17
Max sequence length for outputs: 59


In [9]:
input_token_index = dict(
    [(char,i) for i,char in enumerate(input_characters)])

output_token_index = dict(
    [(char,i) for i,char in enumerate(output_characters)])

In [10]:
input_token_index , output_token_index

({' ': 0,
  '!': 1,
  '"': 2,
  '$': 3,
  '%': 4,
  '&': 5,
  "'": 6,
  ',': 7,
  '-': 8,
  '.': 9,
  '0': 10,
  '1': 11,
  '2': 12,
  '3': 13,
  '4': 14,
  '5': 15,
  '6': 16,
  '7': 17,
  '8': 18,
  '9': 19,
  ':': 20,
  '?': 21,
  'A': 22,
  'B': 23,
  'C': 24,
  'D': 25,
  'E': 26,
  'F': 27,
  'G': 28,
  'H': 29,
  'I': 30,
  'J': 31,
  'K': 32,
  'L': 33,
  'M': 34,
  'N': 35,
  'O': 36,
  'P': 37,
  'Q': 38,
  'R': 39,
  'S': 40,
  'T': 41,
  'U': 42,
  'V': 43,
  'W': 44,
  'Y': 45,
  'Z': 46,
  'a': 47,
  'b': 48,
  'c': 49,
  'd': 50,
  'e': 51,
  'f': 52,
  'g': 53,
  'h': 54,
  'i': 55,
  'j': 56,
  'k': 57,
  'l': 58,
  'm': 59,
  'n': 60,
  'o': 61,
  'p': 62,
  'q': 63,
  'r': 64,
  's': 65,
  't': 66,
  'u': 67,
  'v': 68,
  'w': 69,
  'x': 70,
  'y': 71,
  'z': 72,
  'é': 73},
 {'\t': 0,
  '\n': 1,
  ' ': 2,
  '!': 3,
  '"': 4,
  '$': 5,
  '%': 6,
  '&': 7,
  "'": 8,
  '(': 9,
  ')': 10,
  ',': 11,
  '-': 12,
  '.': 13,
  '0': 14,
  '1': 15,
  '2': 16,
  '3': 17,
  '4'

In [11]:
encoder_input_data = np.zeros(
    (len(input_texts),max_encoder_seq_length,num_encoder_tokens),dtype = 'float32')
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length , num_decoder_tokens), dtype = 'float32')
decoder_output_data = np.zeros(
    (len(input_texts), max_decoder_seq_length,num_decoder_tokens) , dtype = 'float32')

In [12]:
for i,(input_text,output_text) in enumerate(zip(input_texts,output_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i,t,input_token_index[char]] = 1.
    encoder_input_data[i,t+1:,input_token_index[' ']] = 1.
    
    for t, char in enumerate(output_text):
        # decoder_output_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i,t,output_token_index[char]] = 1.
        if t>0:
            # decoder_output_data will be ahead by one timestep
            # and will not include the start character
            decoder_output_data[i,t-1,output_token_index[char]] = 1.
    decoder_input_data[i,t+1:, output_token_index[' ']] = 1.
    decoder_output_data[i , t: , output_token_index[' ']] = 1.

In [13]:
encoder_input_data[0].shape

(17, 74)

In [14]:
# define an input sequence and process it:
encoder_inputs = Input(shape = (None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state= True)
encoder_outputs,state_h,state_c = encoder(encoder_inputs)
# we discard 'encoder_outputs' and only keep the states.
encoder_states = [state_h,state_c]

In [16]:
# set up the decoder , using encoder_states as initial states:
decoder_inputs= Input(shape=(None, num_decoder_tokens))
decoder_lstm = LSTM(latent_dim,return_sequences = True, return_state = True)
decoder_outputs, _ ,_ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens,activation = 'softmax')
decoder_outputs = decoder_dense(decoder_outputs)

In [18]:
# Define the model that will turn
# 'encoder_input_data' & 'decoder_input_data' into 'decoder_output_data'
model = Model([encoder_inputs,decoder_inputs], decoder_outputs)

#Run training:
model.compile(optimizer='rmsprop', loss = 'categorical_crossentropy' , metrics = ['accuracy'])
model.fit([encoder_input_data,decoder_input_data], decoder_output_data,
         batch_size=batch_size,
         epochs = epochs,
         validation_split=0.2)

Epoch 1/100
250/250 [==============================] - 119s 441ms/step - loss: 1.0676 - accuracy: 0.7334 - val_loss: 0.9594 - val_accuracy: 0.7389
Epoch 2/100
250/250 [==============================] - 102s 409ms/step - loss: 0.6946 - accuracy: 0.8029 - val_loss: 0.7539 - val_accuracy: 0.7767
Epoch 3/100
250/250 [==============================] - 105s 418ms/step - loss: 0.5753 - accuracy: 0.8310 - val_loss: 0.6696 - val_accuracy: 0.8028
Epoch 4/100
250/250 [==============================] - 100s 401ms/step - loss: 0.5146 - accuracy: 0.8481 - val_loss: 0.6143 - val_accuracy: 0.8196
Epoch 5/100
250/250 [==============================] - 99s 396ms/step - loss: 0.4724 - accuracy: 0.8595 - val_loss: 0.5870 - val_accuracy: 0.8249
Epoch 6/100
250/250 [==============================] - 100s 400ms/step - loss: 0.4406 - accuracy: 0.8683 - val_loss: 0.5510 - val_accuracy: 0.8349
Epoch 7/100
250/250 [==============================] - 101s 404ms/step - loss: 0.4142 - accuracy: 0.8758 - val_loss: 0.

250/250 [==============================] - 98s 391ms/step - loss: 0.1013 - accuracy: 0.9677 - val_loss: 0.5982 - val_accuracy: 0.8697
Epoch 57/100
250/250 [==============================] - 99s 395ms/step - loss: 0.0996 - accuracy: 0.9682 - val_loss: 0.6038 - val_accuracy: 0.8701
Epoch 58/100
250/250 [==============================] - 98s 393ms/step - loss: 0.0977 - accuracy: 0.9689 - val_loss: 0.6076 - val_accuracy: 0.8694
Epoch 59/100
250/250 [==============================] - 98s 391ms/step - loss: 0.0959 - accuracy: 0.9693 - val_loss: 0.6155 - val_accuracy: 0.8699
Epoch 60/100
250/250 [==============================] - 99s 394ms/step - loss: 0.0939 - accuracy: 0.9699 - val_loss: 0.6259 - val_accuracy: 0.8685
Epoch 61/100
250/250 [==============================] - 98s 391ms/step - loss: 0.0923 - accuracy: 0.9704 - val_loss: 0.6284 - val_accuracy: 0.8693
Epoch 62/100
250/250 [==============================] - 99s 396ms/step - loss: 0.0904 - accuracy: 0.9710 - val_loss: 0.6316 - val_a

In [23]:
# define sampling models:
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape = (latent_dim,))
decoder_state_input_c = Input(shape = (latent_dim,))
decoder_input_states = [decoder_state_input_h,decoder_state_input_c]

decoder_outputs,state_h,state_c = decoder_lstm(decoder_inputs, initial_state=decoder_input_states)
decoder_states = [state_h,state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs]+decoder_input_states,
    [decoder_outputs] + decoder_states)

# Reverse-lookup token index to decode sequences back to something readable.
reverse_input_char_index = dict(
    (i,char) for char,i in input_token_index.items())
reverse_output_char_index = dict(
    (i,char) for char,i in output_token_index.items())

def decode_sequence(input_seq):
    states_value= encoder_model.predict(input_seq)
    output_seq = np.zeros((1,1,num_decoder_tokens))
    output_seq[0,0,output_token_index['\t']] = 1
    
    stop_condition = False
    decoded_sentences = ''
    while not stop_condition:
        output_tokens,h,c = decoder_model.predict(
            [output_seq]+ states_value)
        
        sampled_token_index = np.argmax(output_tokens[0,-1, :])
        sampled_char = reverse_output_char_index[sampled_token_index]
        decoded_sentences += sampled_char

        if(sampled_char == '\n' or len(decoded_sentences) > max_decoder_seq_length):
            stop_condition = True
        
        #update the target sequence (of length 1):
        output_seq = np.zeros((1,1,num_decoder_tokens))
        output_seq[0,0,sampled_token_index] = 1
        
        states_value = [h,c]
    return decoded_sentences

for seq_index in range(20):
    # take one sequence for trying out decoding:
    input_seq = encoder_input_data[seq_index:seq_index+1]
    decoded_sentences = decode_sequence(input_seq)
    print('ProjectGurukul Project: English to French Translation ')
    print('Input sentence:', input_texts[seq_index])
    print('Decoded sentence:' , decoded_sentences)

ProjectGurukul Project: English to French Translation 
Input sentence: Go.
Decoded sentence: Va !

ProjectGurukul Project: English to French Translation 
Input sentence: Go.
Decoded sentence: Va !

ProjectGurukul Project: English to French Translation 
Input sentence: Go.
Decoded sentence: Va !

ProjectGurukul Project: English to French Translation 
Input sentence: Hi.
Decoded sentence: Salut !

ProjectGurukul Project: English to French Translation 
Input sentence: Hi.
Decoded sentence: Salut !

ProjectGurukul Project: English to French Translation 
Input sentence: Run!
Decoded sentence: Cours !

ProjectGurukul Project: English to French Translation 
Input sentence: Run!
Decoded sentence: Cours !

ProjectGurukul Project: English to French Translation 
Input sentence: Run!
Decoded sentence: Cours !

ProjectGurukul Project: English to French Translation 
Input sentence: Run!
Decoded sentence: Cours !

ProjectGurukul Project: English to French Translation 
Input sentence: Run!
Decoded sen